In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,KFold
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings("ignore")

In [11]:
train=pd.read_csv('../data/train_상장_1대1.csv', encoding='euc-kr')
test=pd.read_csv('../data/test_상장_1대1.csv',encoding='euc-kr')

In [12]:
X_train_int=train[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름',
       '자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률']]

X_test_int=test[['부채비율', '총자본회전률', '매출액대비잉여현금흐름', 'PBR', '총자산대비영업현금흐름',
       '자기자본증가율', '총자본투자효율', '총자본순이익률', '매출액영업이익률']]

In [13]:
X_train = train.drop('t-1감사의견코드',axis=1)
y_train = train[['t-1감사의견코드']]

X_test = test.drop('t-1감사의견코드',axis=1)
y_test = test[['t-1감사의견코드']]

In [14]:
X_train_sc = X_train
X_test_sc = X_test

In [15]:
X_train_sum =X_train_sc
X_test_sum =X_test_sc

In [16]:
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from xgboost import XGBClassifier
import numpy as np

def perform_xgb_grid_search(X_train, y_train, k_fold=5):
    # Stratified k-fold cross validation setup
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # XGBoost hyperparameter candidate list setup
    param_grid = {
        'max_depth': [3, 5, 7],
        'n_estimators': [150, 200, 250],
        'learning_rate': [0.01, 0.1, 0.2],
        'objective': ['binary:logistic'],
        'random_state': [0]
    }

    # Initialize an XGBoost classifier
    model = XGBClassifier(use_label_encoder=False)

    # Grid search setup
    grid_search = GridSearchCV(model, param_grid, scoring='f1', cv=cv, verbose=1, n_jobs=-1)

    # Fit the model and tune
    grid_search.fit(X_train, y_train)

    # Print the best hyperparameters
    print("Best Hyperparameters:", grid_search.best_params_)

    # Calculate the average evaluation metric
    mean_f1_score = np.mean(grid_search.cv_results_['mean_test_score'])
    print("Mean F1 Score:", mean_f1_score)

    return grid_search.best_params_, mean_f1_score


In [17]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from xgboost import XGBClassifier
import numpy as np

def evaluate_xgb_with_best_params(X_train, y_train, X_test, y_test, best_params, k_fold=5):
    # Stratified k-fold cross validation setup
    cv = StratifiedKFold(n_splits=k_fold, shuffle=True, random_state=0)

    # Initialize an XGBoost model
    model = XGBClassifier(**best_params, use_label_encoder=False)

    # Lists to save the evaluation metrics for each fold
    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []
    confusion_matrix_list = []

    best_f1_score = 0
    best_model = None

    for fold_idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train), 1):
        X_train_fold, y_train_fold = X_train.iloc[train_idx], y_train.iloc[train_idx]
        X_test_fold, y_test_fold = X_train.iloc[test_idx], y_train.iloc[test_idx]

        # Fit the model
        model.fit(X_train_fold, y_train_fold)

        # Get the predicted probabilities on the test data
        probabilities = model.predict_proba(X_test_fold)

        # Adjust the predicted classes with a threshold of 0.5
        threshold = 0.5
        predicted_classes = (probabilities[:, 1] > threshold).astype(np.int)

        # Calculate the evaluation metrics
        accuracy = accuracy_score(y_test_fold, predicted_classes)
        precision = precision_score(y_test_fold, predicted_classes)
        recall = recall_score(y_test_fold, predicted_classes)
        f1 = f1_score(y_test_fold, predicted_classes)
        conf_matrix = confusion_matrix(y_test_fold, predicted_classes)

        # Append the evaluation metrics for each fold to the lists
        accuracy_list.append(accuracy)
        precision_list.append(precision)
        recall_list.append(recall)
        f1_score_list.append(f1)
        confusion_matrix_list.append(conf_matrix)

        print(f"Fold {fold_idx}")
        print(f"Accuracy: {accuracy}")
        print(f"Precision: {precision}")
        print(f"Recall: {recall}")
        print(f"F1 score: {f1}")
        print("Confusion Matrix:")
        print(conf_matrix)
        print("------------------------------")

        # Save the model with the best F1 score
        if f1 > best_f1_score:
            best_f1_score = f1
            best_model = model

    # Perform the final prediction with the model with the best F1 score
    probabilities_final = best_model.predict_proba(X_test)
    y_pred_final = (probabilities_final[:, 1] > threshold).astype(np.int)

    # Calculate the evaluation metrics
    accuracy_final = accuracy_score(y_test, y_pred_final)
    precision_final = precision_score(y_test, y_pred_final)
    recall_final = recall_score(y_test, y_pred_final)
    f1_final = f1_score(y_test, y_pred_final)
    conf_matrix_final = confusion_matrix(y_test, y_pred_final)

    print("Final Test Results")
    print(f"Accuracy: {accuracy_final}")
    print(f"Precision: {precision_final}")
    print(f"Recall: {recall_final}")
    print(f"F1 score: {f1_final}")
    print("Confusion Matrix:")
    print(conf_matrix_final)

    return accuracy_list, precision_list, recall_list, f1_score_list


In [18]:
best_params, mean_f1_score = perform_xgb_grid_search(X_train, y_train, k_fold=5)
evaluate_xgb_with_best_params(X_train_int, y_train, X_test_int, y_test, best_params, k_fold=5)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 150, 'objective': 'binary:logistic', 'random_state': 0}
Mean F1 Score: 0.8077794881593058
Fold 1
Accuracy: 0.8055555555555556
Precision: 0.7619047619047619
Recall: 0.8888888888888888
F1 score: 0.8205128205128205
Confusion Matrix:
[[13  5]
 [ 2 16]]
------------------------------
Fold 2
Accuracy: 0.8333333333333334
Precision: 0.8
Recall: 0.8888888888888888
F1 score: 0.8421052631578948
Confusion Matrix:
[[14  4]
 [ 2 16]]
------------------------------
Fold 3
Accuracy: 0.8611111111111112
Precision: 0.8421052631578947
Recall: 0.8888888888888888
F1 score: 0.8648648648648649
Confusion Matrix:
[[15  3]
 [ 2 16]]
------------------------------
Fold 4
Accuracy: 0.75
Precision: 0.7368421052631579
Recall: 0.7777777777777778
F1 score: 0.7567567567567567
Confusion Matrix:
[[13  5]
 [ 4 14]]
------------------------------
Fold 5
Accuracy: 0.8888888888888888
Prec

([0.8055555555555556,
  0.8333333333333334,
  0.8611111111111112,
  0.75,
  0.8888888888888888],
 [0.7619047619047619,
  0.8,
  0.8421052631578947,
  0.7368421052631579,
  0.8888888888888888],
 [0.8888888888888888,
  0.8888888888888888,
  0.8888888888888888,
  0.7777777777777778,
  0.8888888888888888],
 [0.8205128205128205,
  0.8421052631578948,
  0.8648648648648649,
  0.7567567567567567,
  0.8888888888888888])